1. [20 pts] In this assignment, we will approach the problem with Word2Vec and contextual analysis of keywords towards sentiment/category processing in our pipeline.
Generate a gensim model of movie reviews. Use any parameters you like while answering the questions (2.) and (3.) below.
Report the size of the vocabulary and characteristics of the gensim model, such as the number of mapping dimensions, etc.

In [1]:
import re
from nltk.corpus import stopwords
import nltk
def ie_preprocess(document):
    document = re.sub('<br />', '', document)
    document = re.sub(r'[^\w\s]', ' ', document)
    sentences = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(document)]
    stop_words = set(stopwords.words('english'))
    return [[word for word in sent if word.lower() not in stop_words] for sent in sentences][0]

In [2]:
import csv
import pandas as pd
path='./movie_data.csv'
df=pd.read_csv(path)

In [3]:
df['review']=df['review'].apply(ie_preprocess)

In [4]:
import nltk
import gensim
print(f'gensim version= {gensim.__version__}')
from gensim.models import Word2Vec

from nltk.corpus import abc

sents = list(df['review'])

model = Word2Vec(df['review'], min_count=2, workers=4)
X = list(model.wv.index_to_key)


print(f'gensim model vocabulary has {len(X)} words mapped to N= {model.vector_size} dimensions')

gensim version= 4.3.2
gensim model vocabulary has 77747 words mapped to N= 100 dimensions


2. [20 pts] Generate the contexts for the following keywords:
(a.) melancholy
(b.) ghastly
(c.) lackluster
(d.) romantic

In [7]:
model.wv.most_similar('melancholy'),

[('wistful', 0.8618030548095703),
 ('powerfully', 0.8534363508224487),
 ('spontaneous', 0.8386438488960266),
 ('sensual', 0.8320813179016113),
 ('lyrical', 0.8318663239479065),
 ('tenderness', 0.8310872912406921),
 ('somber', 0.8268701434135437),
 ('elegance', 0.8260715007781982),
 ('joyous', 0.8244810104370117),
 ('aura', 0.821577250957489)]

In [8]:
model.wv.most_similar('ghastly')

[('spastic', 0.7884919047355652),
 ('transparent', 0.7854024767875671),
 ('rubbery', 0.7850725054740906),
 ('gunwhales', 0.7823657989501953),
 ('thudding', 0.779283881187439),
 ('anachronistic', 0.7777515649795532),
 ('hideously', 0.7773488759994507),
 ('Atmosphere', 0.773285984992981),
 ('phoney', 0.7703019976615906),
 ('abominable', 0.7700594663619995)]

In [9]:
model.wv.most_similar('lackluster')

[('uninspired', 0.8849737644195557),
 ('uneven', 0.8598752617835999),
 ('leaden', 0.8545973300933838),
 ('uninspiring', 0.8521710634231567),
 ('pedestrian', 0.8505023717880249),
 ('stilted', 0.8278570771217346),
 ('abysmal', 0.8236872553825378),
 ('bland', 0.8222702145576477),
 ('dismal', 0.8161261677742004),
 ('lethargic', 0.8146429657936096)]

In [10]:
model.wv.most_similar('romantic')

[('romance', 0.8038605451583862),
 ('screwball', 0.7067616581916809),
 ('tender', 0.6498826146125793),
 ('touching', 0.6463902592658997),
 ('poignant', 0.6307582855224609),
 ('gentle', 0.6267974972724915),
 ('sentimental', 0.6156450510025024),
 ('bittersweet', 0.6069542169570923),
 ('charming', 0.6010169386863708),
 ('sweet', 0.5930174589157104)]

3. [20 pts] Group the reviews into two by the original ground truth of sentiments and repeat question (2.) by generating a positive and a negative gensim model. Report the contexts (4 positive and 4 negative).

In [22]:

sentiment_0=df[df['sentiment'] == 0]['review']
sentiment_1=df[df['sentiment'] == 1]['review']


def different_sentiment(reviews):

    model = Word2Vec(reviews, min_count=2, workers=4)
    X = list(model.wv.index_to_key)


    print(f'gensim model vocabulary has {len(X)} words mapped to N= {model.vector_size} dimensions')
    print(model.wv.most_similar('melancholy'))
    print(model.wv.most_similar('ghastly'))
    print(model.wv.most_similar('lackluster'))
    print(model.wv.most_similar('romantic'))
different_sentiment(sentiment_0)
different_sentiment(sentiment_1)


gensim model vocabulary has 55169 words mapped to N= 100 dimensions
[('Lukas', 0.9403665661811829), ('Celeste', 0.9313252568244934), ('chirpy', 0.9282054901123047), ('Antony', 0.9281792044639587), ('Clifton', 0.9252954721450806), ('Carmen', 0.9250258207321167), ('Peil', 0.9236600995063782), ('Cross', 0.9227039813995361), ('Empress', 0.922613263130188), ('substituting', 0.9225777387619019)]
[('Liotti', 0.9338012933731079), ('tasteful', 0.9292261600494385), ('chintzy', 0.9290462136268616), ('Combine', 0.9285081028938293), ('Samardzic', 0.9284804463386536), ('excessively', 0.9267778396606445), ('EE', 0.9240484237670898), ('cheery', 0.9232284426689148), ('sais', 0.9224592447280884), ('belies', 0.9219242930412292)]
[('leaden', 0.9276651740074158), ('uninspired', 0.9264911413192749), ('uninspiring', 0.9200718998908997), ('uneven', 0.9148015975952148), ('pedestrian', 0.9060354828834534), ('offset', 0.9018392562866211), ('nonexistent', 0.9002106189727783), ('dismal', 0.8993278741836548), ('aby

5. [20 pts] Read the following paper:
Maas, Andrew L., et al. "Learning word vectors for sentiment analysis." Proceedings of the 49th annual meeting of the association for computational linguistics: Human language technologies-volume 1. Association for Computational Linguistics, 2011.
Comment about and/or align your results in this homework.

4. [20 pts] Comment about similarities and differences in (3.). Any comments on why romantic context was not affected?